# Function Programing
<br />
## Higher Order Function
#### Axect

<p style="text-align:center"><b>Refer to "Kansu Programming Jissen Nyumon"</b></p> 

**Table of Contents**

* Higher Order Function
* Reduction

# Higher Order Function

## 지금까지는 개요만 훑었습니다.

---

**지금까지 공부한 것을 나열하면..**

* 함수형 언어가 뭔지, 종류는 뭐가 있는 지, 속도는 좀 나오는지

* 재귀가 뭔지, 추상화, 최적화, 병렬 등 함수형을 왜 써야되는지

* 그 결과, 함수형은 충분히 빠르다는 결론을 얻었습니다.

* 그럼 이제 본격적으로 쓰는 법을 배웁시다

## 손에 물은 묻히지 말자

---

* 재귀는 배웠지만 직접적으로 재귀는 잘 안 씁니다.

> ... "특히 구조 재귀를 직접 작성한다는 것은 때로는 필요하지만, 이는 어셈블러를 직접 쓰는 것과 같은 저급한 행위로 인식된다. 데이터의 구조에 의존하여 그것을 의식한 프로그래밍이 요구되기 때문이다. 이상적으로는 데이터의 구조를 신경 쓰지 않고 전체를 통째로 변환하는 함수만을 조합함으로써 원하는 처리를 작성하고 싶다."

* 실제로 피보나치 400만 이하 짝수만 더하기라는 문제를 함수로 풀 때 우리는 재귀를 직접 짜지 않았습니다.

```haskell
-- Haskell
foldl' (+) 0 $ filter even (takeWhile (<4000000) fibs)
     where fibs = 1:2:(zipWith (+) fibs (tail fibs))
```

```julia
# Julia
fibs = @lazy 1:2:(fibs + drop(1,fibs))
reduce(+, filter(iseven, takewhile(x->x<=4000000, fibs)))
```

**우리가 쓴 거라고는**

* `foldl` 이나 `reduce`

* `filter`

* `takeWhile`

* `zipWith` 나 `drop`

* 항상 함수형 프로그래밍은 위의 함수들에 `map`이라는 함수까지 포함한 함수들만 조합해서 사용

* 즉, 위의 5-6개만 사용할 줄 안다면 함수형 프로그래밍은 50%는 끝남

### Filter

---

* Filter는 조건에 맞는 것만 걸러내주는 함수

* 성능 향상은 없지만 코드 수를 단축시켜주는 고정도르래 역할

```hs
-- Haskell
filter (<3) [4,2,3,5,1]
```

```julia
# Julia
filter(x->x<3, [4,2,3,5,1])
```

**filter의 내부는 다음과 같이 생겼다.**

```hs
filter' :: (a -> Bool) -> [a] -> [a]
filter' p [] = []
filter' p (x:xs) | p x = x : filter' p xs
                 | otherwise = filter' p xs
```

**Quiz : filter로 1에서 100까지 소수만 골라보세요**

* Hint: 소수는 1과 자기자신을 포함하여 약수가 2개인 수입니다.

* Hint2: 소수이면 True, 소수가 아니면 False인 함수를 만드세요.

* Hint3: 약수의 개수를 구하는 함수를 만들어야 소수인지 판별할 수 있습니다.

* Hint4: 약수의 개수를 구하려면 약수를 다 구하는 함수부터 만들어야겠죠?

* 하지만 게으른 언어에서는 Hint3,4와 같이 약수를 구하려 하지 않습니다.

**당신이 filter를 쓰는데 성공했다면 보통 다음과 같을 것입니다.**

```hs
-- Haskell
main = print $ filter(isPrime, [1..100])

isPrime :: Int -> Bool
isPrime n = length (divisors n) == 1

divisors :: Int -> [Int]
divisors n = filter (\x -> mod n x == 0) [1..(n `div` 2)]
```

* 그렇다면 구한 것으로 1212121212가 소수인지 판별해보세요. 아마 오래 걸릴겁니다.

**게으른 언어는 다음과 같이 짤 수 있습니다.**

```hs
-- Haskell (이런 걸 상호재귀 - Mutual Recursion - 이라고 합니다.)
primes = 2:(filter isPrime [3..])
isPrime x = all (\p -> x`mod`p > 0) (takeWhile (\p -> p^2 < x) primes)
```

```julia
# Julia (using Lazy 빼먹지 마세요!)
function isPrime(x)
    infseq = @lazy 3:map(t->t+1,infseq) # 무한수열
    primes = @lazy 2:filter(isPrime,infseq)
    return all(p -> mod(x,p)>0, takewhile(p -> p^2 < x,primes))
end
```

**Quiz: 위에서 설명한 `all` 함수는 어떤 꼴일까요?**

**`all`함수는 다음과 같이 생겼습니다.**

```hs
all' :: (a -> Bool) -> [a] -> Bool
all' _ [] = True
all' p (x:xs) | p x = all p xs
              | otherwise = False
```

* 같은 이치로 `any`함수도 있습니다.

* `all`과 `any`는 상황에 따라 성능향상의 키가 될 수 있으니 유의하시길 바랍니다.

### Map

---

* 구조적 재귀를 회피하고 배열 전체에 연산을 가하거나 타입을 변환할 수 있게 해주는 함수

* `map`이 없으면 함수형 프로그래밍은 존재하기 어렵다. 하지만 성능향상엔 그닥 큰 효과가 없으니 이것도 `filter`처럼 고정도르래 역할이라고 보면 됨

```hs
-- Haskell
map show [1..10]
```

```julia
# Julia
map(Int,zeros(10))
```

**Map은 다음과 같이 생겼습니다.**

```hs
map' :: (a -> b) -> [a] -> [b]
map' _ [] = []
map' f (x:xs) = f x : map' f xs
```

**Quiz: `map`과 `filter`를 모두 사용해서 다음 행렬의 각 행마다 짝수만 고르시오.**

$$
\begin{pmatrix}
1 & 3 & 2 & 4 \\
5 & 1 & 3 & 1 \\
6 & 8 & -1 & 9 \\
10 & 11 & 5 & 8
\end{pmatrix}
$$

* 뭐 다들 쉽게 했을테니 답은 다음과 같습니다.

```hs
-- Haskell
mat = [[1,3,2,4],[5,1,3,1],[6,8,-1,9]]
map (filter even) mat
```

```julia
# Julia
mat = [[1,3,2,4],[5,1,3,1],[6,8,-1,9]]
map(x -> filter(iseven,x), mat)
```

### ZipWith

---

* 함수형에서 항상 나오는 엄청 빠른 피보나치의 메인을 담당하는 함수

* 두 자료형을 동시에 Iteration 하므로 빠른 속도를 보여주는 함수

```hs
-- Haskell
zipWith (+) [1,2] [3,4,5]
fibs = 1:1:(zipWith (+) fibs (tail fibs))
```

```julia
# Julia에는 zipWith가 없지만 무한 수열에 대해서는 + 를 사용할 수 있다.
fibs = @lazy 1:1:(fibs + drop(1,fibs))
```

**`zipWith`는 다음과 같이 생겼습니다.**

```hs
zipWith' :: (a -> b -> c) -> [a] -> [b] -> [c]
zipWith' _ [] _ = []
zipWith' _ _ [] = []
zipWith' f (a:as) (b:bs) = f a b : zipWith' f as bs
```

**Quiz: `zipWith`로 두 행렬 간의 Component 곱을 만들어보세요.**

$$
\begin{pmatrix}
1 & 2 \\
3 & 4
\end{pmatrix} *
\begin{pmatrix}
1 & 2 \\
3 & 4
\end{pmatrix} =
\begin{pmatrix}
1 & 4 \\
9 & 16
\end{pmatrix}
$$

* 좀 어려운 문젠데 놀랍게도 답은 이렇습니다.

```hs
-- Haskell
a = [[1,2],[3,4]]
zipWith (zipWith (*)) a a
```

```julia
# Julia
a = [1 2; 3 4]
a.^2
```

### Foldr / Foldl

---

* 리스트를 접어서 값으로 만들어 드립니다.

* 다른 말로 `reduce`라고도 합니다.

```hs
-- Haskell
foldr (*) 1 [1..5]
foldr (++) "" ["H", "e", "l", "l", "o"]
```

```julia
# Julia
reduce(*, 1:5)
reduce(*,["H", "e", "l", "l", "o"])
```

**`foldr`은 다음과 같이 생겼습니다.**

```hs
foldr' :: (a -> b -> b) -> b -> [a] -> b
foldr' _ e []     = e
foldr' f e (x:xs) = f x (foldr' f e xs)
```

* 생김새를 보면 눈치챌 수도 있겠지만 사실 `foldr`에는 신기한 비밀이 숨겨져있다.

```hs
foldr (:) [] [1..10]
```

* ??!

**`foldr`은 사실 리스트의 `(:)`와 `[]`를 각각 함수와 초기값으로 대체하는 함수였던 것!**

* 무슨 멍멍 소리지?

즉,

```hs
foldr (+) 0 [1..5]
```

에서 `1 : 2 : 3 : 4 : 5 : []` 부분의 `:`를 `+`로, `[]`을 `0`으로 바꾼다는 얘기

```hs
(:) 1 ((:) 2 ((:) 3 ((:) 4 ((:) 5 [])))) => (+) 1 (..((+) 5 0)..)
```

**Quiz: `map` 함수를 `foldr`을 이용하여 나타내시오**

* Hint: `(:)`도 함수임 `->` 합성가능함

* 정답은 바로!

```hs
map'' f = foldr ((:).f) []
```

* 여러모로 `foldr`은 강력한 함수

### Scanr / Scanl

---

* `foldr`이 그냥 접는다면, `scanr`은 하나씩 접습니다.

```hs
scanr (\a b -> a + 10 * b) 4 [1,2,3]
```

* 즉, `foldr`의 중간 과정을 모두 나열하는 것!

**`scanr`은 다음과 같이 생겼습니다.**

```hs
scanr' :: (a -> b -> b) -> b -> [a] -> [b]
scanr' _ e []     = [e]
scanr' f e (x:xs) = f x (head rs) : rs where
    rs = scanr' f e xs
```

**Quiz: `scanr`, `foldr`, `map`, `(.)` 을 모두 사용해서 연속 부분열을 모두 열거하는 함수를 작성하시오**

ex) `segments "abc" = ["a", "b", "c", "ab", "bc", "abc"]` (순서 상관 x)

* Hint: 매우 어려운 문제니 하나씩 생각해보자.

* Hint2: `(\a b -> [a] : map (a:) b)`라는 함수를 생각해보자

# 끝!

### 물론 고차함수만 끝남 ㅋ

# Reduction

## Haskell의 평가 전략

---

* 수학자들이 보기에 `for`문을 써서 일일이 더하는 건 허접해 보이더라

* 계산을 그때그때마다 하는 것은 메모리낭비다. 꼭 필요할 때만 하도록 하자!

* 이러한 평가를 **지연 평가 (Lazy Evaluation)** 이라고 합니다.

* 하지만 아무 이론 없이 '우린 지연 평가로 하겠어' 한다고 되는 것이 아니므로 이론을 세워야 하는데..

## 축약 (Reduction)
---

* 이미 1930년대에 발표된 **Lambda Calculus** 에서 이론은 다 세워짐

* 축약이란 함수 적용의 규칙으로 다음과 같은 규칙을 의미

`(\x -> 변수 x를 포함할지도 모르는 식 A) 식B`

`=>` `식A 안의 변수 x를 모두 식B로 치환한 식`

* 축약이 전부 끝나서 더 이상 축약할 수 없는 식을 **정규형 (normal form)**이라고 한다.

**예를 들어**

```hs
(\x -> x + 1) 2
```

```hs
2 + 1 -- 정규형
```

이렇게 치환하는 규칙!

**그런데 함수가 두 개 이상이라면?**

```hs
(\x -> x + x) ((\x -> x * x) 1)
```

* 음.. 뒤에 것부터 축약해볼까?

```hs
=> (\x -> x + x) (1 * 1)
=> 1 + 1
```

* 이번엔 앞에 부터?

```hs
=> ((\x -> x * x) 1) + ((\x -> x * x) 1)
=> 1 + 1
```

* 눈치가 빠르다면 앞에서의 방법이 적극 평가이고 뒤에서의 방법이 지연 평가라는 것을 눈치챌 수 있다.

### 무한을 축약할 수 있을까?

---

**다음은 축약으로 무한을 만드는 법이다.**

```hs
(\x -> x x) (\x -> x x)
```

위의 식은 어떻게 축약하든 간에 무한반복에 빠지게 된다. 다음과 같은 람다식도 준비해보자.

```hs
(\x -> (\y -> x))
```

이 식은 `x`를 취하면 무엇을 취해도 `x`가 되는 람다식이다.

```hs
   (\x -> (\y -> x)) 1
=> (\y -> 1)
```

**무한 루프와 결합시켜 보자!**

```hs
(\x -> (\y -> x)) 1 ((\x -> x x) (\x -> x x))
```

* 만일 인수부터 축약한다면 당연하게도 무한루프에 빠진다.

* 하지만 앞의 함수부터 축약한다면 답은 바로 1이 나오게 된다.

<p style="text-align:center"><b>즉, 평가 전략으로 어떤 것을 채용하느냐에 따라 계산 또한 많이 달라질 수 있다.</b></p>

### 적극 평가

---

* 적극 평가 (Eager Evaluation) 혹은 정격 평가 (Strict Evaluation)

* 평가할 수 있는 부분을 빨리 평가해 버려, 가능한 한 임 평가가 된 상태로, 더 이상 평가할 필요 없는 값으로 해 놓으려는 평가 전략

* 대부분의 프로그래밍 언어가 적극 평가를 채택하고 있다.

```c
f(g1(), g2(), g3())
```

* 이미 `g1(),g2(),g3()`는 모두 값인 상태인데, 어떤 순서로 실행되는지는 정해지지 않았음.

* 즉, 평가 순서에 의해 버그가 발생할 수 있음. 나쁜 것은 아니지만 일반적으로 코딩할 땐 피해야 함

### 지연 평가

---

* 지연 평가는 평가가 필요로 할 때 까지 평가를 하지 않는 전략

* `Haskell` 정도 만이 상용화된 지연 평가 언어이고 나머지는 부분적으로 도입하거나 도입하지 않음

#### 가장 왼쪽 가장 바깥쪽 축약

---

* 축약의 순서로 바깥쪽에 있는 것부터, 왼쪽에 있는 것부터 축약해 나가는 방법

```hs
(\x -> (\y -> x)) 1 ((\x -> x x) (\x -> x x))
```

* 가장 왼쪽 가장 바깥쪽으로 취하면 1이 나옴

* 수학적으로 가장 왼쪽 가장 바깥쪽 축약으로 정지하지 않는 람다식이라면 다른 어떤 축약 순서를 선택해도 멈추지 않음

* 따라서 정규 순서 (Normal order)라고도 함

#### WHNF

---

* Haskell의 지연 평가에서 식을 평가할 때, 다음의 부분까지 평가한다.
    * 이것 이상 적용할 값이 없는 함수
    * 식의 선두에 생성자가 나온 상태의 값

* 이런 형태를 **WHNF(Weak Head Normal Form)**라고 한다.

* WHNF에서는 람다식의 정규형과 다르게 아직 축약할 부분이 남아 있을지도 모른다.

```hs
isJust :: Maybe a -> Bool
isJust Nothing = False
isJust (Just _) = True
```

* 위의 식에서는 `Just`안에 뭐가 있던지 `True`가 반환되므로 정규형과는 다르다.

* WHNF는 데이터 구조의 평가 타이밍과 그 데이터 구조의 안에 들어 있는 값의 평가 타이밍을 훌륭하게 분리한다.

### Thunk

---

* Haskell의 지연 평가에서는 필요할 때까지 평가를 실시하지 않는 대신에 **평가가 이루어지지 않은 채 방치되고 있는 계산** 또는 **평가가 예정되어 있는 객체**가 그때마다 만들어짐

* 위의 진한 글씨체들을 **성크(Thunk)**라고 함

* Haskell은 필요없거나 역할이 다한 Thunk를 GC의 대상으로 삼아 처리함

### 그래프 축약

---

* Julia가 각광받는 이유는 JIT(Just-In-Time) 컴파일러 때문. 한 번 계산한 것은 캐시로 남아 두 번째는 매우 빠른 속도!

* Haskell은 구현이 어려운 JIT 같은 건 없지만 Thunk를 교묘히 이용함

```hs
foo :: Double
foo = mean [0..9]

bar :: Double
bar = foo + foo
```

* `bar`를 평가하면 함수 `(+)`에 `foo`와 `foo`를 전달하는데, 아직 평가는 하지 않고 `mean [0..9]`를 예정한 Thunk만 만든다.

* 하지만 `(+)`의 계산에서는 `foo` 값이 즉시 필요하므로 `mean [0..9]`가 평가되어 `foo`는 4.5가 된다.

* `foo`가 2개 필요한데, 한 쪽에서 평가된 때에는 `foo`가 4.5로 되어 있고 함수형프로그래밍은 항상 값이 변하지 않으므로 다시 계산하지 않고도 4.5라는 값을 사용할 수 있다.

* 이와 같이 Haskell에서 같은 식을 여러 번 이용하는 것이 분명한 경우, 그 평가가 첫 회에 대해서만 수행되므로 나중에 필요할 때 캐시된 결과를 사용한다. 이것은 Haskell이 순수하다는 성질을 갖고 있고 필요로 하거나 필요로 되는 식끼리의 관계그래프로부터 캐시의 사용 여부를 결정하는 **그래프 축약 (Graph Reduction)**이라는 구조를 가지고 있기 때문

## Eager vs Lazy

---

**적극 평가의 이점, 지연 평가의 결점**

* 적극 평가의 가장 큰 장점은 현재의 컴퓨터 아키텍처와 궁합이 좋다는 것. 애초에 아키텍처가 적극 평가를 기반으로 고려되어 있음

* 가능한 한 계산을 뒤로 미루지 않고 그때 그때 계산하여 값을 만들어 나가므로 복잡한 구조가 필요 없다. 결과적으로 런타임 환경이 매우 단순하다.

* 인간이 직관적으로 떠올리는 실행 순서와 유사하므로 성능이 충분히 예상 가능하다.

* 위의 모든 것에서 말만 부정적으로 바꾸면 지연 평가의 결점이 됨

**지연 평가의 이점, 적극 평가의 결점**

* 지연 평가의 장점 중 하나는 불필요한 계산은 정말로 하지 않음으로써 계산량을 줄일 수 있다는 것

* 명확한 장점은 계산의 정의와 그 실행을 구별할 수 있다는 것

* 위의 장점으로 모듈화가 훨씬 쉬움. 무한한 재귀도 손 쉽게 사용가능

* 하지만 성능 면에서 여전히 물음표는 남을 수 있음

## 평가 제어 = 성능 튜닝!

---

* 베짱이라고 항상 베짱이일 필요는 없다.

* 비장의 무기 `seq`

```hs
Prelude> let loop = loop
Prelude> length [loop, loop, loop]
```

* 위의 코드는 그냥 3이 나오지만

```hs
Prelude> let loop = loop
Prelude> loop `seq` length [loop, loop, loop]
```

* 이 코드는 답이 나오지 않는다.

**다라이 함수 느리게하기**

```hs
tarai :: Int -> Int -> Int -> Int
tarai x y z | x<=y = y
            | otherwise = tarai
                          (tarai (x-1) y z)
                          (tarai (y-1) z x)
                          (tarai (z-1) x y)
```

```hs
tarai' :: Int -> Int -> Int -> Int
tarai' x y z | x<=y = y
             | otherwise = let z' = tarai' (z-1) x y
                           in z' `seq` tarai'
                                       (tarai' (x-1) y z)
                                       (tarai' (y-1) z x)
                                       z'
```

**Summary**

* 적극 평가와 지연 평가는 항상 어느 것이 좋다고 할 수 없는 위치

* 하지만 지연에서 적극은 구현하기 쉬운 반면 애초부터 적극으로 구현된 언어에서 지연을 구현하는 것은 어려움

* 요즘 나온 Scala, Julia 등등의 언어는 지연을 구현하기 쉬운 편에 속하지만 C나 C++은 한숨만..